## Research Questions (in progress - please contact Amelia for any additional questions you would like explored):
1. Financial:
   1. What is the mean/median percentage of expenses spent on program services compared to administrative and fundraising costs? 
      1. Comes from section IX in the form but the IRS extract lacks this specific expense - may need to analyze actual 990 forms from ProPublica’s Non profit Explorer using NLP.
   2. What is the mean/median percentage of revenue comes from program specific services?
   3. What are the primary sources of **<span title="membership dues/fundraising/grants/other">contributions</span>**? 
2. Impact:
   1. What proportion of nonprofits engage in grantmaking, and what are their patterns of distribution?
   2. What is the impact of the programs run by these organizations, and how effectively are they using their funds?
      1. Comes from Section III in the form but the IRS extract lacks this info. What's the best way to look into this?
3. Temporal:
   1. How financially stable are philanthropic organizations over time?

In [ ]:
import plotly.express as px
df = px.data.gapminder()
df = df[df['continent'] != 'Oceania']

# Indicators

## Row {height=60%}


In [ ]:
px.scatter(
  df, x="gdpPercap", y="lifeExp", 
  animation_frame="year", animation_group="country",
  size="pop", color="continent", hover_name="country", 
  facet_col="continent", log_x=True, size_max=45, 
  range_x=[100,100000], range_y=[25,90]
)

## Row {height=40%}


In [ ]:
px.area(df, x="year", y="pop", color="continent", line_group="country")

In [ ]:
px.line(df, x="year", y="lifeExp", color="continent", line_group="country")

# Data

![](gapminder.png) Learn more about the Gapminder dataset at <https://www.gapminder.org/data/documentation/>


In [ ]:
from itables import show
show(df, showIndex = False)